# PyTorch Geometric基础知识
PyG 是一款号称比 DGL 快 14 倍的基于 PyTorch 的几何深度学习框架，可以简单方便的实现图神经网络。

学习PyG的实践代码. [学习资料](https://www.pytorchtutorial.com/pytorch-geometric-for-gnn/#PyTorch_Geometric)

在源教程的基础上稍作改动, 算是更加适合我的版本吧.

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.5.1
True


## Data类
torch_geometric.data 包里有一个 Data 类，通过 Data 类我们可以很方便的创建图结构。

定义一个图结构，需要以下变量：

+ 每个节点（node）的 features
+ 边的连接关系或者边的 features

### Example
表示下面这张图: 一共有四个节点 v1,v2,v3,v4，其中每个节点都有一个二维的特征向量x和一个标签 y。
>20/7/1 猜测x是feature, y是label

![](https://www.pytorchtutorial.com/wp-content/uploads/2020/01/img_5e14b8d64bfa4.png)

1. 表示节点的 features

In [2]:
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

2. 表示边的连接(这里边没有feature)

>通过一组列向量来描述边的连接.

In [3]:
edge_index = torch.tensor([[0,0,1,2,3],
                           [1,3,0,1,2]], dtype=torch.long)

3. 定义图结构

In [4]:
from torch_geometric.data import Data

data = Data(x=x, y=y, edge_index=edge_index)

## Dataset
PyG 里有两种数据集类型：InMemoryDataset 和 Dataset，第一种适用于可以全部放进内存中的小数据集，第二种则适用于不能一次性放进内存中的大数据集。我们以 InMemoryDataset 为例。
> 20/7/1 猜测两种Dataset的API一致

InMemoryDataset 中有下列四个函数需要我们实现：

`raw_file_names()`
返回一个包含所有未处理过的数据文件的文件名的列表。

起始也可以返回一个空列表，然后在后面要说的 process() 函数里再定义。

`processed_file_names()`
返回一个包含所有处理过的数据文件的文件名的列表。

`download()`
如果在数据加载前需要先下载，则在这里定义下载过程，下载到 self.raw_dir 中定义的文件夹位置。

如果不需要下载，返回 pass 即可。

`process()`
这是最重要的一个函数，我们需要在这个函数里把数据处理成一个 Data 对象。

下面是官方的一个示例代码：

In [9]:
from torch_geometric.data import InMemoryDataset
 
 
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(MyOwnDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
 
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
 
    @property
    def processed_file_names(self):
        return ['data.pt']
 
    def download(self):
        # Download to `self.raw_dir`.
        pass
 
    def process(self):
        # Read data into huge `Data` list.
        data_list = [...]
 
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]
 
        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
 
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

本文接下来会介绍如何用 RecSys Challenge 2015 的数据创建一个自定义数据集。

## DataLoader
就是使用原始Pytorch中的Dataloader

## MessagePassing
Message Passing 是图网络中学习 node embedding 的重要方法。

Message Passing 的公示如下：

![](https://www.pytorchtutorial.com/wp-content/uploads/2020/01/img_5e14c0ce52bab.png)

其中，x 表示表格节点的 embedding，e 表示边的特征，ϕ 表示 message 函数，□ 表示聚合 aggregation 函数，γ 表示 update 函数。上标表示层的 index，比如说，当 k = 1 时，x 则表示所有输入网络的图结构的数据。

下面是每个函数的介绍：

`propagate(edge_index, size=None, **kwargs)`
这个函数最终会调用 message 和 update 函数。

`message(**kwargs)`
这个函数定义了对于每个节点对 (xi,xj)，怎样生成信息（message）。

`update(aggr_out, **kwargs)`
这个函数利用聚合好的信息（message）更新每个节点的 embedding。

### 示例：SageConv
我们来看看怎样实现论文 “Inductive Representation Learning on Large Graphs” 中的 SageConv 层。SageConv 的 Message Passing 定义如下：

![](https://www.pytorchtutorial.com/wp-content/uploads/2020/01/img_5e14c3404305b.png)

聚合函数（aggregation）我们用最大池化（max pooling），这样上述公示中的 AGGREGATE 可以写为：

![](https://www.pytorchtutorial.com/wp-content/uploads/2020/01/img_5e14c39149141.png)

上述公式中，对于每个邻居节点，都和一个 weighted matrix 相乘，并且加上一个 bias，传给一个激活函数。相关代码如下：


In [11]:
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max')
        # for message
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        # for update
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]
 
        x_j = self.lin(x_j)
        x_j = self.act(x_j)
      
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]

        new_embedding = torch.cat([aggr_out, x], dim=1)
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)

        return new_embedding

# 示例：RecSys Challenge 2015
RecSys Challenge 2015 是一个挑战赛，主要目的是创建一个 session-based recommender system。主要任务有两个：

1. 预测经过一系列的点击后，是否会产生购买行为。
2. 预测购买的商品。

数据下载地址在[这里](https://www.pytorchtutorial.com/goto/https://2015.recsyschallenge.com/challenge.html)。数据主要包含两部分：yoochoose-clicks.dat 点击数据, 和 yoochoose-buys.dat 购买行为数据。

## 数据预处理

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('data/yoochoose-data/yoochoose-clicks.dat', header=None)
df.columns=['session_id','timestamp','item_id','category']
 
buy_df = pd.read_csv('data/yoochoose-data/yoochoose-buys.dat', header=None)
buy_df.columns=['session_id','timestamp','item_id','price','quantity']

item_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id)

df.head()

,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,2053,0
1,1,2014-04-07T10:54:09.868Z,2052,0
2,1,2014-04-07T10:54:46.998Z,2054,0
3,1,2014-04-07T10:57:00.306Z,9876,0
4,2,2014-04-07T13:56:37.614Z,19448,0


对数据随机抽样（数据太多了）

In [ ]:
import numpy as np
sampled_session_id = np.random.choice(df.session_id.unique(), 1000000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()